# Réseaux de neurones unrolled pour l'IRM

Les lignes suivantes permettent d'installer une bibliothèque pour le calcul de la Transformée de Fourier Non-Uniforme (NUFFT).

In [ ]:
!git clone https://github.com/albangossard/Bindings-NUFFT-pytorch
![ -e nufftbindings/ ] && rm -r -f nufftbindings/
!mv Bindings-NUFFT-pytorch/nufftbindings/ ./
!rm -r -f Bindings-NUFFT-pytorch/

Téléchargement des données

In [ ]:
!pip install gdown
!gdown https://drive.google.com/uc?id=17k1CYZ4bgbv6q4T4q_zSmEFhwcWlDSVZ
!tar -xzf fastMRI.tar.gz
!rm -r -f data/fastMRI/
!mv fastMRI/ data/fastMRI/
!rm fastMRI.tar.gz

### Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
# from tensorboardX import SummaryWriter

from DIDN import DIDN
import nufftbindings.kbnufft as nufft
import dataLoaderfastMRI
import scripts.metrics as metrics
from scripts.recon import cg

In [ ]:
nx = ny = 320
Nbatch = 8

device = torch.device('cuda:0')

xi = torch.tensor(np.load("data/xi_10.npy")).to(device)
print(xi.shape, xi.dtype)
K = xi.shape[0]

nufft.nufft.set_dims(K, (nx, ny), device, Nb=Nbatch)

nufft.nufft.precompute(xi)

In [ ]:
dataset_train = dataLoaderfastMRI.fastMRIdatasetKnee(train=True)
dataset_test = dataLoaderfastMRI.fastMRIdatasetKnee(train=False)
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=Nbatch, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=Nbatch, shuffle=True)
print('nb images in training dataset:',len(dataset_train))
print('nb images in testing dataset:',len(dataset_test))

Ecrire une fonction réalisant l'optimisation du modèle sur une epoch.

Coder une fonction testant le modèle sur tout le jeu de données test et qui renvoie un array numpy des PSNR associés à chaque image.

Implémenter une fonction appelant $Nepoch$ fois la fonction train et test. Ne pas oublier d'appeler un éventuel scheduler passé en argument.

## Reconstructeur adjoint

## Unrolled forward-backward

## Unrolled ADMM